<a href="https://colab.research.google.com/github/CaP4iK/cretes/blob/main/%D0%98%D1%81%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BE%D1%88%D0%B8%D0%B1%D0%BE%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Устанавливаем pyenchant


In [16]:
!apt install enchant

Reading package lists... Done
Building dependency tree       
Reading state information... Done
enchant is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.


In [17]:
!pip install pyenchant

Импорт

In [18]:
import enchant  # при импроте пишем именно enchant (не pyenchant)   
dictionary = enchant.Dict("en_US")
print(dictionary.check("driver"))

True


Если вывел True, значит верно

Допустим ошибку в слове driver (опечатались, например)

In [19]:
print(dictionary.check("draiver"))

False


Получили False ? - значит в библиотеке такое слово не нашлось, а значит лучше поправить его, как?

Выведем предложения по исправлению

In [20]:
print(dictionary.suggest(u"draiver"))

['driver', 'drainer', 'Rivera']


Пора добавить Русский язык

In [21]:
!apt update
!apt install hunspell-ru

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [22]:
dictionary = enchant.Dict("ru_RU")

Проверим правописание на русском языке

In [23]:
print(dictionary.check("Водитель"))

True


In [24]:
print(dictionary.check("Водиьель"))

False


In [28]:
print(dictionary.suggest(u"Водиьель"))

['Водитель']


difflib, который позволяет сравнивать строковые последовательности. Попробуем выбрать из списка слово «Водитель»:

In [31]:
import enchant
import difflib

woi = "Водтель" 
sim = dict() 

#dictionary = enchant.Dict("en_US")
dictionary = enchant.Dict("ru_RU")
suggestions = set(dictionary.suggest(woi))

for word in suggestions:
    measure = difflib.SequenceMatcher(None, woi, word).ratio()
    sim[measure] = word

print("Возможно вы имели ввиду:", sim[max(sim.keys())])

Возможно вы имели ввиду: Водитель


В словаре sim будут храниться значения степеней сходства (диапазон от 0 до 1) предложенных методом suggest() класса Dict слов с искомым словом («draiver»). Данные значения мы получаем в цикле при вызове метода ratio() класса SequenceMatcher и записываем в словарь. В конце получаем слово, которое максимально близко к проверяемому.

Выше мы работали с отдельными словами, но будет полезно разобраться, как работать с целыми блоками текста. Для этой задачи нужно использовать класс SpellChecker:

In [32]:
from enchant.checker import SpellChecker

checker = SpellChecker("ru_RU")
checker.set_text("Я хосу научиться водмть.")
print([i.word for i in checker])

['хосу', 'водмть']


в итоге мы поймали неверно написанные слова (слова содержащие ошибки)

Как видно, это не сложнее работы с отдельными словами. Кроме того, класс SpellChecker предоставляет возможность использовать фильтры, которые будут игнорировать особые последовательности, не являющиеся ошибочными, например, адрес электронной почты. Для этого необходимо импортировать класс или классы фильтров, если их несколько, и передать список фильтров параметру filters классу SpellChecker:

In [45]:
from enchant.checker import SpellChecker
from enchant.tokenize import EmailFilter, URLFilter

checker_with_filters = SpellChecker("en_US", filters=[EmailFilter])
checker_with_filters.set_text("Приает! My neim is John and thiz is my email: johnnyhatesjazz@gmail.com.")
print([i.word for i in checker_with_filters])

['Приает', 'neim', 'thiz']


Отлично, а что делать если текст русский а в нем встечаются слова на English?

In [36]:
de = enchant.Dict("en_US")
dr = enchant.Dict("ru_RU")

de.check("Hellp")


False

In [42]:
a = "Привнт"
dr.check(a)

False

In [43]:
print(dr.suggest(a))

['Привит', 'Приват', 'Привет', 'Привинтит']


In [44]:
print(de.suggest(a))

[]


+/- так можно понять какой язык, 